In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# Data Load
forest = gpd.read_file("../data/processed/전국_임상도_전처리.gpkg")
sig = gpd.read_file("../data/processed/대한민국_시군구.gpkg")

In [3]:
forest.head(5)

,수종코드,수종,면적,시도코드,geometry
0,1,침엽수,8804.462978,29,"MULTIPOLYGON (((923284.835 1681929.385, 923285..."
1,1,침엽수,8672.386988,29,"MULTIPOLYGON (((924732.298 1679316.902, 924732..."
2,1,침엽수,6283.905748,29,"MULTIPOLYGON (((923699.238 1682110.512, 923701..."
3,1,침엽수,78185.887476,29,"MULTIPOLYGON (((923457.864 1680852.092, 923455..."
4,1,침엽수,33279.489729,29,"MULTIPOLYGON (((923241.573 1680378.461, 923241..."


In [6]:
sig.head(5)

,시군구코드,시군구명,면적,시도코드,시도명,geometry
0,11110,종로구,2.397161e+07,11,서울특별시,"MULTIPOLYGON (((956615.453 1953567.199, 956621..."
1,11140,중구,9.962768e+06,11,서울특별시,"MULTIPOLYGON (((957890.386 1952616.746, 957909..."
2,11170,용산구,2.189756e+07,11,서울특별시,"MULTIPOLYGON (((953115.761 1950834.084, 953114..."
3,11200,성동구,1.680078e+07,11,서울특별시,"MULTIPOLYGON (((959681.109 1952649.605, 959842..."
4,11215,광진구,1.702881e+07,11,서울특별시,"MULTIPOLYGON (((964825.058 1952633.25, 964875...."


In [7]:
# 좌표계 통일
forest = forest.to_crs(sig.crs)

In [8]:
# 공간 조인
joined = gpd.sjoin(forest, sig, how="inner", predicate="intersects")
joined["면적"] = joined.geometry.area

In [9]:
# 구별 수종 비율 계산 후 저장
summary = joined.groupby(["시군구명", "수종"])["면적"].sum().unstack(fill_value=0)
summary["총면적"] = summary.sum(axis=1)
for kind in ["침엽수", "활엽수", "혼효림", "기타"]:
    if kind not in summary.columns:
        summary[kind] = 0
    summary[f"{kind}_비율"] = (summary[kind] / summary["총면적"]) * 100
result = summary[[f"{k}_비율" for k in ["침엽수", "활엽수", "혼효림", "기타"]]].round(2).reset_index()

In [10]:
result.head(5)

수종,시군구명,침엽수_비율,활엽수_비율,혼효림_비율,기타_비율
0,가평군,28.10,61.48,7.79,2.63
1,강남구,13.91,75.22,8.12,2.76
2,강동구,9.99,79.95,9.34,0.72
3,강북구,10.48,60.93,23.66,4.92
4,강서구,36.51,43.94,16.39,3.16


In [11]:
# 혼효림 반반 나누기
result["침엽수_tmp"] = result["침엽수_비율"] + result["혼효림_비율"] * 0.5
result["활엽수_tmp"] = result["활엽수_비율"] + result["혼효림_비율"] * 0.5

# 전체 합계 (기타는 제외)
total = result["침엽수_tmp"] + result["활엽수_tmp"]

# 정규화: 100% 기준 재비율화
result["침엽수_비율"] = (result["침엽수_tmp"] / total) * 100
result["활엽수_비율"] = (result["활엽수_tmp"] / total) * 100

# 반올림
result[["침엽수_비율", "활엽수_비율"]] = result[["침엽수_비율", "활엽수_비율"]].round(2)

# 불필요한 컬럼 제거
result = result.drop(columns=["혼효림_비율", "기타_비율", "침엽수_tmp", "활엽수_tmp"], errors='ignore')

In [12]:
result.head(5)

수종,시군구명,침엽수_비율,활엽수_비율
0,가평군,32.86,67.14
1,강남구,18.48,81.52
2,강동구,14.77,85.23
3,강북구,23.47,76.53
4,강서구,46.16,53.84


In [13]:
# 결과 저장
result.to_csv("../data/processed/시군구별_수종비율.csv", index=False, encoding="utf-8-sig")